In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=161d1dfc64d5051538b93946da63231ccfdb7aca0daf71c17d3e653e0eb34aa1
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = (
    SparkSession.builder
      .appName("Exercicios Spark")
      .getOrCreate())

In [5]:
!./ngrok authtoken cr_2bPPeoC6NJqG0Huey7fRe713uFd
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

caminho_csv = "./base_de_dados.csv"

spark.read.csv(
      path=caminho_csv,
      header=True,
      sep=";",
      schema=schema_base_pix,
      timestampFormat="dd/MM/yyyy HH:mm"
).createOrReplaceTempView('base_pix')

In [7]:

#1. Descubra a quantidade total de transações na base de dados

spark.sql(

          'select count(*) as numero_transacoes from base_pix'
).show()

+-----------------+
|numero_transacoes|
+-----------------+
|              100|
+-----------------+



In [8]:

# 2. Qual foi a quantia total de dinheiro transacional em cada ano?

spark.sql('''
              select
                year(data_transacao),
                sum(valor)
              from base_pix
              group by 1
''').show()


+--------------------+------------------+
|year(data_transacao)|        sum(valor)|
+--------------------+------------------+
|                2022|517980.58999999973|
|                2021| 848320.6000000001|
+--------------------+------------------+



In [9]:

# 3.Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.

spark.sql('''
            select
              sum(valor)
            from base_pix
            where parte_creditada_banco = 'Nubank'
''').show()

+----------+
|sum(valor)|
+----------+
| 302224.64|
+----------+



In [10]:
# 4. Retorne a quantidade de transações agrupadas por tipo de chave.

spark.sql('''
              select
              count(*),
              chave_pix_tipo
              from base_pix
              group by 2
              order by 1 desc

''').show()

+--------+--------------+
|count(1)|chave_pix_tipo|
+--------+--------------+
|      49|           cpf|
|      29|         email|
|      22|       celular|
+--------+--------------+



In [11]:
# 5. Descubra qual é a média, mínima e máxima do valor de todas as transações.

spark.sql('''
              select
              min(valor),
              avg(valor),
              max(valor)
              from base_pix


''').show()


+----------+------------------+----------+
|min(valor)|        avg(valor)|max(valor)|
+----------+------------------+----------+
|      0.03|13663.011899999998|  95977.62|
+----------+------------------+----------+



In [12]:
# 6. Quais foram as 5 maiores transações durante todo o período?

spark.sql('''
              select *
              from base_pix
              order by valor desc
              limit 5


''').show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|     chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| 54|95977.62|   Henrique Ferreira|             6269315|              Nubank|Sra. Maria Luiza ...|             68542779|                  BTG|           cpf|         14957860211|2021-09-22 10:17:00|
| 86|94736.79|Joao Vitor Cavalc...|            52265189|                 BTG|        Stella Gomes|             93858790|               Nubank|         email|stella.gomes@yaho...|2021-01-26 00:04:00|
| 97|

In [13]:
# 7. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações.

spark.sql('''
              select chave_pix_tipo, count(*)
              from base_pix
              where parte_debitada_banco = 'BTG' OR parte_creditada_banco = 'BTG'
              group by 1
              order by 2 desc


''').show()

+--------------+--------+
|chave_pix_tipo|count(1)|
+--------------+--------+
|           cpf|      22|
|         email|      15|
|       celular|      13|
+--------------+--------+



In [20]:
# 8. Descubra qual foi a maior transação de cada mês na base de dados.

spark.sql('''

    with cte (
            SELECT
                  id,
                  valor,
                  date_format(data_transacao, 'y-MM') as ano_mes,
                  row_number() over (
                            partition by date_format(data_transacao, 'y-MM')
                          order by valor desc
                                   ) as row_number
          FROM base_pix
          ) select
        id,
        valor,
        ano_mes
      from cte
      where row_number = 1
          ''').show()


+---+--------+-------+
| id|   valor|ano_mes|
+---+--------+-------+
| 86|94736.79|2021-01|
| 78|81977.98|2021-02|
| 88|78347.58|2021-03|
| 44|   459.2|2021-04|
| 68|  611.53|2021-05|
| 52|60139.23|2021-06|
| 76| 9320.87|2021-07|
| 73|  570.52|2021-08|
| 54|95977.62|2021-09|
| 32| 1508.83|2021-10|
| 71|80083.34|2021-11|
| 40|  829.87|2021-12|
| 79| 78559.4|2022-01|
| 35| 7944.02|2022-02|
| 55|35409.61|2022-03|
| 31|20875.64|2022-04|
| 47|38219.08|2022-05|
| 36|48714.95|2022-06|
| 18|49836.72|2022-07|
| 12| 4374.56|2022-08|
+---+--------+-------+
only showing top 20 rows



In [26]:

# 9. Crie uma nova coluna que contenha o valor da transação em dólar.
spark.sql('''
              select id, valor, round(valor/4.93, 2) as valor_dolar
              from base_pix


''').show()

+---+--------+-----------+
| id|   valor|valor_dolar|
+---+--------+-----------+
|  1|    9.93|       2.01|
|  2|   15.38|       3.12|
|  3|   57.58|      11.68|
|  4|53705.13|   10893.54|
|  5|25299.69|    5131.78|
|  6| 7165.06|    1453.36|
|  7|    6.16|       1.25|
|  8|  136.36|      27.66|
|  9|  574.39|     116.51|
| 10|   42.88|        8.7|
| 11|33629.97|    6821.49|
| 12| 4374.56|     887.33|
| 13|  507.18|     102.88|
| 14|67758.87|   13744.19|
| 15|  815.53|     165.42|
| 16|    2.73|       0.55|
| 17|    0.54|       0.11|
| 18|49836.72|   10108.87|
| 19|    9.68|       1.96|
| 20| 9837.22|    1995.38|
+---+--------+-----------+
only showing top 20 rows

